In [1]:
import time
from time import sleep
import struct
import serial
from serial import Serial
from serial.tools.list_ports import comports
from datetime import datetime
print(datetime.now())

import pose_openvr_wrapper
import numpy as np
import rtde_control
import rtde_receive
from numpy.linalg import norm
from scipy.spatial.transform import Rotation as R
from pyrr import quaternion as qv
import robotiq_gripper
import copy

2024-10-24 15:37:10.871962


In [2]:
for port in comports():
    print(port)

COM3 - Silicon Labs CP210x USB to UART Bridge (COM3)


In [3]:
# Для маятника
port = Serial("COM3", 230400, timeout=1)
port.isOpen()

True

In [4]:
pyopenvr_wrapper = pose_openvr_wrapper.OpenvrWrapper('config.json')
print(pyopenvr_wrapper.devices)

rtde_r = rtde_receive.RTDEReceiveInterface("192.168.1.110")
rtde_c = rtde_control.RTDEControlInterface("192.168.1.110")

gripper = robotiq_gripper.RobotiqGripper()
gripper.connect("192.168.1.110", 63352)
if not gripper.is_active():
    gripper.activate()

{'tracker_0': {'name': 'tracker_0', 'type': 'tracker', 'serial': 'LHR-B6119ABB', 'index': 4}}


## Teleoperation

### Functions

In [5]:
velocity = 0.5
acceleration = 0.5
dt = 1.0/500  # 2ms
lookahead_time = 0.2
gain = 100

In [6]:
# Joint limits
# Elbow = 0.5 2.0
# Wrist1 = -0.7 0.8 
# Wrist2 = 0.8 2.5
# Wrist3 = 504.4 507.4

In [7]:
def isInJointLimits(q):
    if q[2]<0.2 or q[2]>2.0:
        return False
    if q[3]<-0.7 or q[3]>1.4:
        return False
    if q[4]< 0.0 or q[4]> 2.5:
        return False
    if q[5]< 506.5 or q[5]> 511.5:
        return False
    return True

def filter(hand, old_hand, pos_tol = 0.01, angle_tol = 0.02):
    pos_vec = np.zeros(3)
    pos_vec[0] = hand['x'][0] - old_hand['x'][0]
    pos_vec[1] = hand['z'][0] - old_hand['z'][0]
    pos_vec[2] = hand['y'][0] - old_hand['y'][0]

    if norm(pos_vec) > pos_tol:
        return False
    
    q = [hand[axis][0] for axis in ['r_x', 'r_y', 'r_z', 'r_w']]
    q_old = [old_hand[axis][0] for axis in ['r_x', 'r_y', 'r_z', 'r_w']]
    dif = [a - b for a,b in zip(q, q_old)]
    if norm(dif) > angle_tol:
        return False

    return True
    
def quaternion_multiply(quaternion1, quaternion0, reverse = False):
    if reverse:
        x0, y0, z0, w0 = quaternion0
        x1, y1, z1, w1 = quaternion1
    else:
        w0, x0, y0, z0 = quaternion0
        w1, x1, y1, z1 = quaternion1

    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

def hand_to_ur(hand, base_hand, base_robot, sens = 1.0):
    ur = np.zeros(6)
    vec = np.array([0., 0., 0.0819])

    # position
    ur[0] = base_robot[0] - (hand['x'][0] - base_hand['x'][0])/sens
    ur[1] = base_robot[1] + (hand['z'][0] - base_hand['z'][0])/sens
    ur[2] = base_robot[2] + (hand['y'][0] - base_hand['y'][0])/sens
    
    # orientation
    qh0 = [base_hand[axis][0] for axis in ['r_x', 'r_y', 'r_z', 'r_w']]
    q = [hand[axis][0] for axis in ['r_x', 'r_y', 'r_z', 'r_w']]
    q_inv = qv.inverse(q)
    qh = quaternion_multiply(q_inv, qh0, reverse = True)

    qh_init = copy.deepcopy(qh)
    qh[2] = -qh_init[3]
    qh[1] = qh_init[2]
    qh[3] = qh_init[1]
    qh[0] = -qh_init[0]

    rot = R.from_rotvec(base_robot[3:6])
    qr0 = rot.as_quat()
    qr = quaternion_multiply(qh, qr0)

    ur[3:6] = R.from_quat(qr).as_rotvec()

    # positional offset
    rot2 = R.from_quat(qr)
    base_vec = rot.apply(vec)
    new_vec = rot2.apply(vec)
    dif = new_vec - base_vec
    ur[0:3] += dif
    
    return ur

In [8]:
def crc32mpeg2(data):
    crc = 0xFFFFFFFF
    for byte in data:
        crc ^= byte << 24
        for _ in range(8):
            if crc & 0x80000000:
                crc = (crc << 1) ^ 0x04C11DB7
            else:
                crc <<= 1
            crc &= 0xFFFFFFFF
    return crc

def read_pose():
    flag_message_error = False
    package = bytearray(b"i")
    port.flushInput()
    port.write(package)
    # Получаем данные из порта
    data = port.read(8)
    # Проверяем, что данные получены и их размер равен 8 байт
    if data and len(data) == 8:
        teleop_position = np.frombuffer(data[:4], dtype=np.uint32)[0]
        CRC_STM = np.frombuffer(data[4:], dtype=np.uint32)[0]
        # high-endian это очень важно (!)
        bytes_for_crc = bytearray(struct.pack(">I", teleop_position))
        crc32_python = crc32mpeg2(bytes_for_crc)
        if crc32_python != CRC_STM:
            flag_message_error = True
        else:
            teleop_position = round(teleop_position / 3)
            if teleop_position >= 255:
                teleop_position=254
            if teleop_position == 33:
                teleop_position=0
            return teleop_position
    else:
        flag_message_error = True
        teleop_position=0
        return teleop_position

### Start

In [12]:
# Base pose
rtde_c.servoJ([1.7286394834518433,
 -1.5598433653460901,
 1.5497121810913086,
 0.05845451354980469,
 1.6997259855270386,
 508.9877680103683], velocity, acceleration, dt, lookahead_time, gain)

True

: 

In [10]:
base_hand = pyopenvr_wrapper.sample('tracker_0', samples_count=1)
base_robot = rtde_r.getActualTCPPose()
print(base_hand)
print(base_robot)

{'time': [0.0], 'x': [0.5255894660949707], 'y': [1.2190396785736084], 'z': [0.31392526626586914], 'r_x': [0.5998512969784721], 'r_y': [0.18723424143872103], 'r_z': [-0.2796167853764243], 'r_w': [0.7259036353107409], 'roll': [74.65510774794413], 'pitch': [37.39345537822029], 'yaw': [-13.192832606409223], 'matrix': [array([[ 0.77351534,  0.63057506, -0.06362896,  0.52558947],
       [-0.1813243 ,  0.12398575, -0.9755761 ,  1.21903968],
       [-0.60728502,  0.76616085,  0.21024325,  0.31392527],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])]}
[0.1455136087790952, -0.27437987620450827, 0.3090733704548076, 1.618471882450375, -0.011796131454891333, 0.05319128094562374]


In [11]:
# old_hand = pyopenvr_wrapper.sample('tracker_0', samples_count=1)
base_hand = pyopenvr_wrapper.sample('tracker_0', samples_count=1)
base_robot = rtde_r.getActualTCPPose()

while(1):
    hand = pyopenvr_wrapper.sample('tracker_0', samples_count=1)
    # if filter(hand, old_hand):
    #     old_hand = hand.copy() 
    ur_pose = hand_to_ur(hand, base_hand, base_robot)
    if rtde_c.isPoseWithinSafetyLimits(ur_pose):
        q = rtde_c.getInverseKinematics(ur_pose)
        # print(q)
        if isInJointLimits(q):
            rtde_c.servoJ(q, velocity, acceleration, dt, lookahead_time, gain)
        
    # print('Has solution: ', rtde_c.getInverseKinematicsHasSolution(ur_pose, rtde_r.getActualQ()))
    # print(ur_pose[3:6])

    pos = read_pose()
    if (pos < gripper._max_position - 30) and (pos > gripper._min_position + 30):
        gripper.move(pos, 100, 10)
        time.sleep(0.01)
    # time.sleep(0.002)

KeyboardInterrupt: 

In [17]:
while(1):
    pos = read_pose()
    if (pos < gripper._max_position - 30) and (pos > gripper._min_position + 30):
        gripper.move(pos, 100, 10)
        time.sleep(0.01)

KeyboardInterrupt: 

In [22]:
read_pose()

70